# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  6.06it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.67it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.34it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.36it/s]



  0%|          | 0/23 [00:00<?, ?it/s]

...

 74%|███████▍  | 17/23 [00:04<00:01,  4.67it/s]

 78%|███████▊  | 18/23 [00:04<00:01,  4.65it/s]

 87%|████████▋ | 20/23 [00:05<00:00,  4.82it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  4.99it/s]

100%|██████████| 23/23 [00:05<00:00,  3.84it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Travis and I am a senior at the University of Southern California. I am currently studying business with a focus on finance. I am excited to start this internship program with American Express and I am looking forward to learning more about the financial industry.
As a student, I have been involved in several extracurricular activities including being a member of the Trojan Investment Club, where I work alongside other students to manage a portfolio of stocks and bonds. I have also been involved in a few entrepreneurial ventures, including starting a small e-commerce business with a friend.
I am excited to learn more about the financial industry and gain experience in a professional setting.
Prompt: The president of the United States is
Generated text:  about to take a very big step into the world of climate change, one that has the potential to make a significant dent in America’s contributions to global warming.
President Joe Biden is set to

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Tom

,

 and

 I

...

 transparency

 in

 its

 development

.

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Chris, I am a second-year MSc student at the University of Warwick studying a course called Business and Management. I have lived in Warwick since 2017 and I love the campus and the university environment. I am an international student from Malaysia and I am excited to share my experiences and perspectives on studying in the UK.
I have always been fascinated by the world of business and management, and I chose this course to broaden my knowledge and skills in this field. As a Malaysian student, I was attracted to the UK's high standard of education and the diversity of students at the university. I am enjoying my time at the University of Warwick

Prompt: The capital of France is
Generated text:  Paris. The city of Paris is situated on the Seine River, in the northern part of the country. The population of Paris is approximately 2.2 million people. The city is one of the most visited cities in the world, with millions of tourists visiting eac

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Emma

.

 I

 am

...

 to

 understand

 and

 respond

 to

In [6]:
llm.shutdown()